In [1]:
import primefac
import multiprocess
import collections
from timeout import Timeout
import itertools
import operator
import functools
import random
import subprocess
import sys
sys.set_int_max_str_digits(13000)

In [2]:
a,b=1,1
x,y=[1],[1]
for _ in range(10000):
  a,b=8*a+7*b,9*a+8*b
  x.append(a)
  y.append(b)
a_list=x.copy()
b_list=y.copy()

In [3]:
def read_primes():
  with open('primes.txt', 'r') as file:
    primes = set(map(int, file.read().split()))
  # print('read', len(primes), 'primes')
  return primes
def write_primes(primes):
  for filename in ['primes.txt', 'new_primes.txt']:
    with open(filename, 'a') as file:
      file.write('\n'.join([str(val) for val in primes])+'\n')
  print(f'added {len(primes)} primes\n', end='', flush=True)
def rewrite_primes(primes):
  with open('primes.txt', 'w') as file:
    file.write('\n'.join([str(val) for val in primes])+'\n')
  print(f'rewrote {len(primes)} primes\n', end='', flush=True)
def syncprimes(primes):
  ops = read_primes()
  df = set(primes).difference(ops)
  if df:
    write_primes(df)


In [4]:
def read_factors():
  with open('factors.txt', 'r') as file:
    all_factors = [[int(numstr) for numstr in line.split(',')[1:]] for line in file.read().split()]
  return all_factors[:10001],all_factors[10001:]
def rewrite_factors(a_factors,b_factors):
  with open('factors.txt', 'w') as file:
    file.write('\n'.join([','.join([str(i)]+list(map(str, factors[i]))) for factors in [a_factors,b_factors] for i in range(len(factors))])+'\n')
  print('rewrote factors')
def sync_factors(primes, a_factors, b_factors):
  for i in range(10001):
    a = a_list[i]//functools.reduce(operator.mul, a_factors[i], 1)
    b = b_list[i]//functools.reduce(operator.mul, b_factors[i], 1)
    x[i],y[i] = a,b
    for v,vf,arr in [(a,a_factors,x), (b,b_factors,y)]:
      if v==1: continue
      for p in primes:
        if v%p: continue
        while v%p == 0:
          v //= p
          arr[i] //= p
          vf[i].append(p)
        if v == 1: break
        if v in primes:
          vf[i].append(v)
          arr[i] //= v
          v //= v
          break
def sync_factors_at_indices(primes, indices, a_factors, b_factors):
  for p, i in zip(primes, indices):
    a = a_list[i]//functools.reduce(operator.mul, a_factors[i], 1)
    b = b_list[i]//functools.reduce(operator.mul, b_factors[i], 1)
    x[i],y[i] = a,b
    for v,vf,arr in [(a,a_factors,x), (b,b_factors,y)]:
      if v==1: continue
      if v%p: continue
      while v%p == 0:
        v //= p
        arr[i] //= p
        vf[i].append(p)
      if v == 1: break
      if v in primes:
        vf[i].append(v)
        arr[i] //= v
        v //= v
        break
def sync_new_factors():
  with open('new_primes.txt', 'r') as file:
    new_primes = set(map(int, file.read().split()))
  new_primes = [p for p in new_primes if primefac.isprime(p)]
  sync_factors(new_primes, a_factors, b_factors)
  rewrite_factors(a_factors,b_factors)
  with open('new_primes.txt', 'w') as file:
    file.write('')


In [5]:
a_factors,b_factors = read_factors()
primes = read_primes()
global_primes = primes.copy()
invalis = set([1,2,6])
valps = set([0,1,2,4])
negis = set()
posis = set()
ff = set()

In [6]:
# for i in range(10001):
#   for af in a_factors[i]:
#     while x[i]%af == 0:
#       x[i] //= af
#   for bf in b_factors[i]:
#     while y[i]%bf == 0:
#       y[i] //= bf

In [7]:
# for i,v in enumerate(x):
#   if i%200 == 0:
#     print(i)
#   if v != 1:
#     if primefac.isprime(v) and v not in primes:
#       print(i, v)
#       syncprimes([v])
#     # for p in primes:
#     #   while v%p==0:
#     #     print(v,p)
#     #     v //= p


In [8]:
sync_new_factors()

rewrote factors


In [9]:
dd = collections.Counter([i for v in [a_factors,b_factors] for r in v for i in r])
print(dd)

ddwr = collections.Counter([i for v in [a_factors,b_factors] for r in v for i in set(r)])
print(sorted([(v,k) for k,v in ddwr.items()])[::-1])

ddcwr = collections.Counter([i for av,bv in zip(a_factors,b_factors) for i in set(av).union(bv)])
print(ddcwr==ddwr)

Counter({3: 5001, 5: 4167, 17: 3543, 239: 2008, 271: 2007, 7: 1667, 13: 1548, 293: 1434, 617: 1431, 19: 1172, 71: 1127, 4049: 1111, 419: 911, 2309: 909, 1097009: 909, 131: 775, 117701: 769, 17483311: 769, 31: 690, 13709: 667, 1951: 667, 101: 594, 38775469: 588, 4440692161: 588, 911: 527, 68513089: 526, 70772438609: 526, 41: 488, 17410177: 476, 374681: 476, 47: 444, 139: 438, 352589: 435, 323471: 435, 1289: 435, 3313: 435, 89561: 435, 18677801: 400, 56599: 400, 4751: 400, 222510401: 400, 22110582149: 370, 159839: 370, 414991: 370, 59: 351, 2957: 345, 367837541873: 345, 2765207: 345, 26315036393: 345, 61: 328, 2330950146013: 323, 7193: 323, 10621646561: 323, 109182559: 323, 263: 304, 9769: 303, 14519: 303, 7919: 303, 3768285047: 303, 349: 287, 817646467189: 286, 4129: 286, 60950142049: 286, 73: 274, 4140124084452621921391: 270, 224901512375431: 270, 285937: 270, 859: 256, 1637: 256, 202881431: 256, 19679858951: 256, 12791: 256, 83: 247, 20769906956429479: 244, 609997: 244, 2297: 244, 108

In [10]:
def factor_with_timeout(i, num, timeout, ps, name, es=True):
  ev = False
  try:
    with Timeout(timeout):
      while not (ev and es) and num>1:
        p = primefac.ecm(num)
        print(f'{name}{i} p {p}\n', end='', flush=True)
        syncprimes([p])
        num //= p
        ps.append(p)
        if p%7 not in valps:
          ev=(True, False, str(p)+' | '+name)
          negis.add(i)
          break

  except Timeout.Timeout:
    pass
  return num, ev

In [11]:
def chk(i, ecmtimes, primes, es=True):
  if es and i%7 in invalis:
    negis.add(i)
    return (True, False, 'invalid i')
  if es and i in negis:
    return (True, False, 'prev result')
  if es and i in posis:
    return (True, True, 'prev result')
  aps, bps = [], []
  ev = False
  # if es:
  #   a,b = x[i],y[i]
  # else:
  a,b = a_list[i], b_list[i]
  for p in a_factors[i]:
    while a%p == 0:
      a //= p
      aps.append(p)
      if p%7 not in valps:
        ev=(True, False, str(p)+' | a')
        negis.add(i)
        if es: break
    if (ev and es) or a==1: break

  for p in b_factors[i]:
    while b%p == 0:
      b //= p
      bps.append(p)
      if p%7 not in valps:
        ev=(True, False, str(p)+' | b')
        negis.add(i)
        if es: break
    if (ev and es) or b==1: break

  if not (ev and es) and (a > 1 or b > 1):
    if ecmtimes: print(f'starting: {i}\n', end='', flush=True)
    for timeout in ecmtimes:
      # print('t=',timeout, end=' ',sep='')
      a,nev = factor_with_timeout(i,a,timeout,aps,'a',es)
      ev = nev if not ev else ev
      
      if (ev and es) or (a==1 and b==1):
        break

      b,nev = factor_with_timeout(i,b,timeout,bps,'b',es)
      ev = nev if not ev else ev
      
      if (ev and es) or (a==1 and b==1):
        break
  
  x[i], y[i] = a, b

  if ev:
    return ev

  if (a==1 and b==1):
    print(f"Positive! {i} {aps} {bps}\n", end='', flush=True)
    posis.add(i)
    return (True, True, 'a='+'*'.join(map(str,aps))+'\nb='+'*'.join(map(str,bps)))
  else:
    print(f"Unknown: {i} {aps} {bps}\n", end='', flush=True)
    return (False, False, (True, True, 'a='+'*'.join(map(str,aps))+'\nb='+'*'.join(map(str,bps))))


In [12]:
def zipr(rng):
  return zip(rng,[x[i] for i in rng],[y[i] for i in rng])
times = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]
def sim_thread(v, es=True):
  res = chk(v, times[8:11], read_primes(), es)
  return res
def sim_thread_full(v):
  return sim_thread(v, es=False)


In [13]:
def refill_negis():
  for ecmtimes in [[]]:# + [[t] for t in times]:
    for i in range(10001):
      chk(i,ecmtimes,primes)
  print(len(set(range(10001)).difference(negis).difference(posis)),posis)
  print([len([i for i in range(idx*1000,idx*1000+1000) if i not in negis|posis]) for idx in range(10)])
refill_negis()

Positive! 0 [] []
Positive! 26 [17231429089624614166470862182959] [19538604045167506118097869511631]
Positive! 33 [1607, 243402458839, 11504689616265565750737583] [45590256356873, 407359, 274750586941086808567]
Positive! 35 [69206257, 16515714544563093108617538971911153] [569, 2313398660689, 12497, 14767, 5335242610482412601]
Unknown: 320 [] [14405679222159815567849704896529]
Unknown: 495 [] [91346417, 1236769]
Unknown: 530 [27560537, 98093990579209, 679039] []
Unknown: 543 [] [2788833159490513]
Unknown: 761 [30267300658319] [1218401, 17316531492570919759361]
Unknown: 831 [3059921, 80929863860619087920068943721623] [107522927, 5920279, 87854873162768304289409310594517066841]
Unknown: 938 [15017, 24552609684838893048633344948071] [8823516892849]
Unknown: 1118 [18700578632212732690875068811332461262066345639509526350549755168867545441371750164351826305301179194487533578345174490289245349791010555322087183244958284103490092995517369326762388357911192436624301231936952097712648481094913247

In [14]:
pool = multiprocess.Pool(6)
outs = pool.map(sim_thread, set(range(200,320)).difference(negis))
outs

[]

In [15]:
pool = multiprocess.Pool(4)
outs = pool.map(sim_thread_full, range(100))
[o for o in outs if not o[0]]

Positive! 0 [] []
Positive! 26 [17231429089624614166470862182959] [19538604045167506118097869511631]
Positive! 35 [69206257, 16515714544563093108617538971911153] [569, 2313398660689, 12497, 14767, 5335242610482412601]
Positive! 33 [1607, 243402458839, 11504689616265565750737583] [45590256356873, 407359, 274750586941086808567]


[]

In [16]:
pool = multiprocess.Pool(4)
# outs = pool.map(sim_thread, random.sample(list(range(7000,8000)),250))

In [17]:
pool = multiprocess.Pool(4)
# outs = pool.map(sim_thread, random.sample(list(range(5000,10001)),5001))

In [18]:
todo = set(range(2001)).difference(negis).difference(posis)
print(len(todo))
sorted(todo)

28


[320,
 495,
 530,
 543,
 761,
 831,
 938,
 1118,
 1188,
 1223,
 1251,
 1260,
 1355,
 1356,
 1383,
 1398,
 1400,
 1404,
 1463,
 1505,
 1559,
 1650,
 1664,
 1685,
 1785,
 1790,
 1923,
 1958]

In [19]:
def sync_check_primes(primes):
  # print(len(primes))
  primes = primes.union([])
  syncprimes(primes)
  primes = read_primes()
  for p in list(primes):
    if p not in global_primes and not primefac.isprime(p):
      print(p)
      primes.remove(p)
  rewrite_primes(primes)
  return primes
def resync_all(primes):
  print(len(primes))
  primes = sync_check_primes(primes)
  sync_new_factors()
  refill_negis()
  return primes
primes = resync_all(primes)

16252
rewrote 16252 primes
rewrote factors
Unknown: 320 [] [14405679222159815567849704896529]
Unknown: 495 [] [91346417, 1236769]
Unknown: 530 [27560537, 98093990579209, 679039] []
Unknown: 543 [] [2788833159490513]
Unknown: 761 [30267300658319] [1218401, 17316531492570919759361]
Unknown: 831 [3059921, 80929863860619087920068943721623] [107522927, 5920279, 87854873162768304289409310594517066841]
Unknown: 938 [15017, 24552609684838893048633344948071] [8823516892849]
Unknown: 1118 [18700578632212732690875068811332461262066345639509526350549755168867545441371750164351826305301179194487533578345174490289245349791010555322087183244958284103490092995517369326762388357911192436624301231936952097712648481094913247052890113329851039762919620103662298993753877671389607266921409589642120122426985451793588184895538245205751878071153500014936713778268662358088253035492610148202416761386895011788541234704335190673026899235775229066794866543323208458837164355268333536620318932145948163996191747110398

In [20]:
for i in range(10001):
  if divmod(a_list[i],functools.reduce(operator.mul, a_factors[i], 1))!=(x[i],0):
    print(i)
  if divmod(b_list[i],functools.reduce(operator.mul, b_factors[i], 1))!=(y[i],0):
    print(i)

In [21]:
for i in range(100):
  if x[i] != 1:
    print(x[i])
    s = subprocess.run(['../cado-nfs/cado-nfs.py', str(x[i])], capture_output=True)
    print('=', s.stdout)
    primes = primes.union(list(map(int, s.stdout.split())))
  if y[i] != 1:
    print(y[i])
    s = subprocess.run(['../cado-nfs/cado-nfs.py', str(y[i])], capture_output=True)
    print('=', s.stdout)
    primes = resync_all(primes.union(list(map(int, s.stdout.split()))))


In [22]:
#https://github.com/stevenhalim/cpbook-code/blob/master/ch5/primes.py

_sieve_size = 0
bs = []
_primes = []


def sieve(upperbound):
  global _sieve_size, bs, _primes

  _sieve_size = upperbound+1
  bs = [True] * 1000000010
  bs[0] = bs[1] = False
  for i in range(2, _sieve_size):
    if bs[i]:
      for j in range(i*i, _sieve_size, i):
        bs[j] = False
      _primes.append(i)


def isPrime(N):
  global _sieve_size, _primes
  if N <= _sieve_size:
    return bs[N]
  for p in _primes:
    if p * p > N:
      break
    if N % p == 0:
      return False
  return True


def primeFactors(N):
  global _primes

  factors = []
  for p in _primes:
    if p * p > N:
      break
    while N % p == 0:
      N //= p
      factors.append(p)
  if N != 1:
    factors.append(N)

  return factors


def numPF(N):
  global _primes

  ans = 0
  for p in _primes:
    if p * p > N:
      break
    while N % p == 0:
      N //= p
      ans += 1
  if N != 1:
    ans += 1

  return ans


def numDiffPF(N):
  global _primes

  ans = 0
  for p in _primes:
    if p * p > N:
      break
    if N % p == 0:
      ans += 1
    while N % p == 0:
      N //= p
  if N != 1:
    ans += 1

  return ans


def sumPF(N):
  global _primes

  ans = 0
  for p in _primes:
    if p * p > N:
      break
    while N % p == 0:
      N //= p
      ans += p
  if N != 1:
    ans += N

  return ans


def numDiv(N):
  global _primes

  ans = 1
  for p in _primes:
    if p * p > N:
      break
    power = 0
    while N % p == 0:
      N //= p
      power += 1
    ans = ans * (power + 1)
  if N != 1:
    return 2 * ans
  else:
    return ans


def sumDiv(N):
  global _primes

  ans = 1
  for p in _primes:
    if p * p > N:
      break
    multiplier = p
    total = 1
    while N % p == 0:
      N //= p
      total += multiplier
      multiplier *= p
    ans *= total
  if N != 1:
    ans *= N+1

  return ans


def EulerPhi(N):
  global _primes

  ans = N
  for p in _primes:
    if p * p > N:
      break
    if N % p == 0:
      ans -= ans // p
    while N % p == 0:
      N //= p
  if N != 1:
    ans -= ans // N

  return ans


def main():
  global _primes

  sieve(1000000000)
#main()

In [23]:
try:
  idx = 0
  inlist = []
  stv = 50847533
  for idx,p in enumerate(_primes[stv:]):
    if p in primes:
      continue
    # if p > 25000000:
    #   break
    aa,bb = 1,1
    i = 0
    while aa*bb and i < 10000:
      i += 1
      aa,bb = (8*aa+7*bb)%p, (9*aa+8*bb)%p
    if (not aa*bb):
      print(" " if i in negis else "!", i, p)
      inlist.append(p)
      if len(inlist) >= 300:
        primes = resync_all(primes.union(inlist))
        inlist = []
        print(stv+idx)
      # sync_factors_at_indices(p, range(i,10002,i*2+1), b_factors)
      # rewrite_factors(a_factors,b_factors)

  if inlist:
    primes = resync_all(primes.union(inlist))
    inlist = []
    print(stv+idx)
except:
  primes = resync_all(primes.union(inlist))
  inlist = []
  print(stv+idx)

In [24]:
# for i in range(10001):
#   if i in negis: continue
#   if a_factors[i] and primefac.isprime(x[i]):
#     print('a',x[i])
#   if b_factors[i] and primefac.isprime(y[i]):
#     print('b',y[i])
#   print(f'done {i}', flush=True)